# Opening and cleaning out ephys data

Note: Use L1imag/formation environment.

## Setup everything

### Import packages

In [ ]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import fnmatch
import os
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
%matplotlib widget
from ipyfilechooser import FileChooser

from ephyviewer import mkQApp, MainViewer, TraceViewer

### Define general functions

In [ ]:
# Function to find files containing a specific string
def find_files_with_string(folder_path, search_string):
    matching_files = []
    # Traverse the folder to find files
    for root, _, files in os.walk(folder_path):
        for file in files:
            if fnmatch.fnmatch(file, f"*{search_string}*"):
                matching_files.append(os.path.join(root, file))
    return matching_files

### Choose experiment

In [ ]:
dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"
try:
    %store -r dpath
except:
    print("data not in store")
    dpath ="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

# Set up Initial Basic Parameters#
minian_path = "."

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>OpenEphys Folder</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

## Load data
### Load timestamps from continuous recording

Note here the synchronised timestamps are not loaded. Attempt is made next cell but there is a bug because the files are of different lengths. 
What needs to be done is to understand the exact content of these files to then load them properly.

In [ ]:

folder = Path('.').absolute()
print(folder)
path_list_ERS = []
Ephys_rec_stamps = {}

for file in folder.glob('**/*continuous/Rhythm_FPGA-112.0/timestamps.npy'):
    path_list_ERS.append(file)

for file_path in folder.glob('**/*continuous/Rhythm_FPGA-112.0/timestamps.npy'):
    recording = file_path.parents[2].stem
    arr = np.load(file_path)
    Ephys_rec_stamps[recording] = arr

## Here the timestamps are stored in a dict which is not necessarily what I want, maybe will need to amend that

    


In [ ]:
# Not working yet as synchronised timestamps and timestamps for recording 2 are of different size.

TTL_stamp2 = []
for file_path in folder.glob('**/*.npy'):
    subfolder = file_path.parents[1].stem
    if subfolder == 'continuous':
        recording = file_path.parents[2].stem.replace('recording','')
        print(recording)
        file = file_path.stem
        print(recording, file)
        np_arr = np.load(file_path)
        datalen = len(np_arr)
        print(file, datalen)
        if recording == 1: #not in TTL_stamp2:
            TTL_stamp2.append(recording)
            coords = {
                'channels' : np.array(['synchronized_timestamps', 'timestamps']),
                'duration_rec' : np.arange(datalen)
            }
            globals()[f"StampsCont_{recording}"] = xr.DataArray(coords=coords, dims=['channels', 'duration_rec'])
        globals()[f"StampsCont_{recording}"].loc[file,:] = np_arr   


### Load timestamps for miniscope frames and laser (TTL in)


In [ ]:

TTL_stamps = []
list_recordings = []
for file_path in folder.glob('**/*.npy'):
    subfolder = file_path.parents[0].stem
    if subfolder == 'TTL_1':
        recording = file_path.parents[3].stem.replace('recording','')
        file = file_path.stem
        np_arr = np.load(file_path)
        datalen = len(np_arr)
        if recording not in TTL_stamps:
            TTL_stamps.append(recording)
            list_recordings.append(file_path.parents[3].stem)
            coords = {
                'channels' : np.array(['full_words', 'timestamps', 'channel_states', 'channels']),
                'duration_rec' : np.arange(datalen)
            }
            globals()[f"Allstamps_{recording}"] = xr.DataArray(coords=coords, dims=['channels', 'duration_rec'])
        globals()[f"Allstamps_{recording}"].loc[file,:] = np_arr   




### Load recordings

#### Merge files

In [ ]:
folderpath = Path(dpath)
numchannels=32

if find_files_with_string(folderpath,  ".bin"): #Bonsai or IgorPro
    recSyst = "Bonsai" #"IgorPro"
    matching_files = find_files_with_string(folderpath, ".bin")
    fileType="OE32channels.bin"

elif find_files_with_string(folderpath,  "continuous.dat"): #OpenEphys
    recSyst = "OpenEphys" #"IgorPro"
    matching_files = find_files_with_string(folderpath, "continuous.dat")
    fileType="OE32channels.bin"

match recSyst:
    case "Bonsai":
        dtype=np.uint16
        offset=int(65535/2)
    case _:
        dtype=np.int16
        offset=0

All = np.zeros([0], dtype=dtype)
for file in matching_files:
    print("importing {}".format(file))
    file_data = np.fromfile(file, dtype=dtype)
    All=np.append(All, file_data, axis=0)
if offset != 0:
    All = All - offset
if All.dtype is not np.dtype(np.int16):
    All = All.astype(np.int16)
All = All.reshape(-1,numchannels)

print('File loaded: {}, with {} channels and {} datapoint'.format(fileType, numchannels, All.shape[0] ))

### Distribute channels.

EMG = 11, deep PFC = 17, superficial PFC = 18, deep S1 = 24, superficial S1 = 23, deep CA1 = 21, superficial CA1 = 20

WARNING: this has to be adjusted for every mouse. In the future, add a cell at the beginning to enter these parameters

In [ ]:
sampling_rate = 25000 # Hz

# Distribute
EMG = All[:,10]
PFC = All[:,17] #- All[:,17]
S1 = All[:,22] - All[:,23] 
CA1 = All[:,20]- All[:,19]

# Stack back
combined = np.stack((EMG, PFC, S1, CA1), axis = 1)

coords = {
    'brain_areas' : np.array(['EMG', 'PFC', "S1", "CA1"]),
    'duration_rec' : np.arange(0, combined.shape[0]/sampling_rate, 1/sampling_rate)
}

# Put in xarray
xrCombined = xr.DataArray(coords=coords, dims=['duration_rec', 'brain_areas'])
xrCombined.loc[:,:]  = combined

# Save datas
combinedFN = os.path.join(dpath,'RawDataChannelExtracted.npy')
np.save(combinedFN, combined)


#  Downsample all signals to 1 kz.


In [ ]:
combined = All[:100000]
new_sampling_rate = 1000 # Hz

newLen = int(combined.shape[0] * new_sampling_rate / sampling_rate)
print(combined.shape[0], newLen)
combinedDS = signal.resample(combined, newLen, axis = 0)
#combinedDS = signal.decimate(combined, 25, axis = 0)
print('resampled', combinedDS.shape)
combinedDSFN = os.path.join(dpath,'RawDataChannelExtractedDS.npy')
np.save(combinedDSFN, combinedDS)

## End of notebook. 
Data is cleaned up and saved. Data processing for different cortical areas on specific notebooks. Next step is WakeRemoving notebook.

Below is for quick filtering, plotting and visualisation to assess data quality. Filtering for data processing is done again in specific notebooks.

Data quality is assessed on a Sample, whose value has to be attributed on the initial cell.

## Filtering 

SWR: 120 - 200 Hz

In [ ]:
Sample = filtered_GoodChannels_1

f_CA1 = Sample[:, 3].copy()

# Paramètres de notre filtre :
f_lowcut = 120.
f_hicut = 200.
fs = new_sampling_rate
nyq = 0.5 * fs
N = 6                 # Ordre du filtre
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
print(Wn)

# Création du filtre :
b, a = scipy.signal.butter(N, Wn, 'band')
filt_SWR_CA1 = scipy.signal.filtfilt(b, a, f_CA1)

times = np.arange(0, f_CA1.size/new_sampling_rate, 1./new_sampling_rate)



Spindles: 8 - 16 Hz

In [ ]:
f_PFC = Sample[:, 1].copy()
f_S1 = Sample[:, 2].copy()

# Paramètres de notre filtre :
f_lowcut = 10.
f_hicut = 16.
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
N = 4
# Création du filtre :
b, a = scipy.signal.butter(N, Wn, 'band')
filt_Spind_PFC = scipy.signal.filtfilt(b, a, f_PFC)
filt_Spind_S1 = scipy.signal.filtfilt(b, a, f_S1)

# # Calcul de la reponse en fréquence du filtre
# w, h = signal.freqz(b, a)

# # Tracé de la réponse en fréquence du filtre
# fig, ax = plt.subplots(figsize=(8,5)) 

# ax.plot(0.5*fs*w/np.pi, np.abs(h), 'b')

# ax.set_xlabel('frequency [Hz]')
# ax.set_ylabel('Amplitude [dB]')
# ax.grid(which='both', axis='both')

FFT display

In [ ]:
# Calcul du spectre
f, Pxx_den = signal.welch(f_CA1, fs, nperseg=1024)

# Tracé
fig, ax = plt.subplots(figsize=(10,5)) 
ax.semilogy(f, Pxx_den)   #  plot with log scaling on the y axis
ax.set_xlabel('frequency [Hz]')
ax.set_ylabel('PSD [V**2/Hz]')

In [ ]:
plt.close('all')

# Display. 

Can massively be improved: with Matplotlib

In [ ]:
times_sliced = times[000:200000]
filt_to_display = filt_SWR_CA1[000:200000]-1000
f_CA1_sliced = f_CA1[0000:200000]/2
combined = np.stack((f_CA1_sliced, filt_to_display), axis = 1)
# Tracé du signal filtré

fig, ax = plt.subplots(1,1, figsize=(15,5), layout='constrained') 
ax.plot(times_sliced, combined, 'r')
ax.set_xlabel('Temps [sec]')
ax.set_ylabel('Amplitude')


With ephyviewer. 

In [ ]:
# prepare numpy array for ephyviewer

filt_SWR_CA1_sliced= filt_SWR_CA1[000:200000, np.newaxis]
filt_Spind_PFC_sliced= filt_Spind_PFC[000:200000, np.newaxis]
filt_Spind_S1_sliced= filt_Spind_S1[000:200000, np.newaxis]
filt_sliced = Sample[0000:200000,:]
combined2 = filt_sliced[:,0:2].copy()
intf_sliced = filt_sliced[:,2]
intf_sliced = intf_sliced[:, np.newaxis]
combined2 = np.append(combined2, filt_Spind_PFC_sliced, axis=1)
combined2 = np.append(combined2, intf_sliced, axis=1)
combined2 = np.append(combined2, filt_Spind_S1_sliced, axis=1)
intf_sliced = filt_sliced[:,3]
intf_sliced = intf_sliced[:, np.newaxis]
combined2 = np.append(combined2, intf_sliced, axis=1)
combined2 = np.append(combined2, filt_SWR_CA1_sliced, axis = 1)


In [ ]:
app = mkQApp()


sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined2, sample_rate, t_start, 'Signals')
win.add_view(view1)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#aa0000'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#0055ff'
view1.by_channel_params['ch5', 'color'] = '#ff5500'
view1.by_channel_params['ch6', 'color'] = '#ff5500'

view1.by_channel_params['ch0', 'gain'] = 0.00002
view1.by_channel_params['ch1', 'gain'] = 0.00002
view1.by_channel_params['ch2', 'gain'] = 0.00004
view1.by_channel_params['ch3', 'gain'] = 0.0001
view1.by_channel_params['ch4', 'gain'] = 0.0001
view1.by_channel_params['ch5', 'gain'] = 0.00002
view1.by_channel_params['ch6', 'gain'] = 0.00005

view1.by_channel_params['ch0', 'offset'] = 0.3
view1.by_channel_params['ch1', 'offset'] = 0.2
view1.by_channel_params['ch2', 'offset'] = 0.1
view1.by_channel_params['ch3', 'offset'] = 0.
view1.by_channel_params['ch4', 'offset'] = -0.1
view1.by_channel_params['ch5', 'offset'] = -0.2
view1.by_channel_params['ch6', 'offset'] = -0.3

#Run
win.show()
app.exec_()
